# Introduction

## Goal
The goal of this lab is to familiarize yourself with Particle Swarm Optimization and study the effect of parametrization on the algorithmic performance.

Note once again that, unless otherwise specified, in this module's exercises we will use real-valued genotypes and that the aim of the algorithms will be to *minimize* the fitness function $f(\mathbf{x})$, i.e. lower values correspond to a better fitness!

### Helper Code

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pygame as pg
from pygame import Vector2, Surface
from typing_extensions import Self
from typing import Any
import random


class Rules:
    def __init__(self, screen_width: int, screen_height: int):
        self.screen_width = screen_width
        self.screen_height = screen_height

    ###############
    ### Boid rules
    ###############

    # Find the neighbors of a boid in range
    def neighbors(self, boids: list[Any]) -> list[Any]:
        neighbors: list[Any] = []
        for boid in boids:
            if boid.position != self.position:
                if self.position.distance_to(boid.position) < self.radius:  # type: ignore
                    neighbors.append(boid)
        return neighbors

    # Cohesion, boids try to fly towards the center of mass of neighboring boids
    def fly_towards_center(self, boids: list[Any]) -> Vector2:
        center = Vector2(0, 0)
        for boid in boids:
            if boid.position != self.position:
                center += boid.position
            return (center - self.position) / 100
        return Vector2(0, 0)

    # Separation, boids try to keep a small distance away from other objects (boids, obstacles)
    def keep_distance_away(self, boids: list[Any], range: int = 9) -> Vector2:
        distance = Vector2()
        for boid in boids:
            if boid.position != self.position:
                if (boid.position - self.position).length() < range:
                    distance = distance - (boid.position - self.position)
        return distance

    # Alignment, boids try to match velocity with nearby boids
    def match_velocity(self, boids: list[Any]) -> Vector2:
        velocity = Vector2(0, 0)
        for boid in boids:
            if boid.position != self.position:
                velocity += boid.velocity
        if len(boids) > 1:
            velocity = velocity / (len(boids) - 1)
            return (velocity - self.velocity) / 8  # type: ignore
        return Vector2(0, 0)

    # Avoid the hoiks
    def tend_to_place(self, hoiks: list[Any]) -> Vector2:
        run = Vector2(0, 0)
        for hoik in hoiks:
            if (hoik.position - self.position).length() < self.radius:  # type: ignore
                run = -(hoik.position - self.position).normalize()
            if (hoik.position - self.position).length() < hoik.size:  # type: ignore
                # Respawn object if colided with other object
                self.position = Vector2(
                    random.uniform(0, self.screen_width),
                    random.uniform(0, self.screen_height),
                )
        return run

    ###############
    ### Hoik rules
    ###############

    # Separation, hoiks try to keep distance away from eachother
    def my_food(self, hoiks: list[Any]):
        distance = Vector2()
        for hoik in hoiks:
            if hoik.position != self.position:
                if (hoik.position - self.position).length() < 50:
                    distance = distance - (hoik.position - self.position)
        return distance

    # Chase the boids in range
    def chase(self, boids: list[Any]):
        closest_boid = None
        for boid in boids:
            distance = abs(self.position.distance_to(boid.position))
            if distance < self.radius:  # type: ignore
                closest_boid = boid
        if closest_boid:
            return (closest_boid.position - self.position).normalize()

        return Vector2(0, 0)

    #########################
    ### Screen wrap or bound
    #########################

    # Wrap the boids around the screen when they go off screen
    def wrap_position(self):
        if self.position.x > self.screen_width:
            self.position.x = 0
        if self.position.x < 0:
            self.position.x = self.screen_width
        if self.position.y > self.screen_height:
            self.position.y = 0
        if self.position.y < 0:
            self.position.y = self.screen_height

    # Bound the boids to the screen. Change the velocity when they reach margin
    def bound_position(self, margin: int = 100):
        if self.position.x > self.screen_width - margin:
            self.velocity += Vector2(-0.7, 0)  # type: ignore
        if self.position.x < margin:
            self.velocity += Vector2(0.7, 0)  # type: ignore
        if self.position.y > self.screen_height - margin:
            self.velocity += Vector2(0, -0.7)  # type: ignore
        if self.position.y < margin:
            self.velocity += Vector2(0, 0.7)  # type: ignore


class Boid(Rules):
    def __init__(self, screen_width: int, screen_height: int):
        super().__init__(screen_width, screen_height)
        self.position = Vector2(
            random.uniform(0, screen_width), random.uniform(0, screen_height)
        )
        self.velocity = Vector2(random.uniform(-1, 1), random.uniform(-1, 1))
        self.radius = 100

    # Draw the boid
    def draw(self, screen: Surface):
        pg.draw.circle(screen, "red", (int(self.position.x), int(self.position.y)), 5)

    ### Update the position of the boid
    def update(
        self, boids: list[Self], ALIGNMENT: float, COHESION: float, SEPARATION: float
    ):
        ### Weights of the rules

        ### Neighbors in range
        n = Rules.neighbors(self, boids)  # type: ignore

        ### Rules to follow
        alignment = ALIGNMENT * Rules.match_velocity(self, n)
        cohesion = COHESION * Rules.fly_towards_center(self, n)
        separation_from_boids = SEPARATION * Rules.keep_distance_away(self, n, 9)

        # Update velocity
        self.velocity += alignment + cohesion + separation_from_boids

        # Limit the speed of the boids
        self.velocity.scale_to_length(5)

        # Update position
        self.position += self.velocity

        # Wrap the position of the boids
        Rules.bound_position(self)


sys.path.insert(0, "")

WIDTH = 800
HEIGHT = 800


def run(WIDTH: int, HEIGHT: int, num_boids: int, a: float, c: float, s: float):
    pg.init()
    screen = pg.display.set_mode((WIDTH, HEIGHT))
    screen.fill("#FFFAFA")
    pg.display.set_caption("Boids")
    clock = pg.time.Clock()

    # Create boids, hoiks and obstacles
    boids = [Boid(WIDTH, HEIGHT) for _ in range(num_boids)]

    running = True
    while running:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                running = False
        screen.fill((255, 250, 250))

        # Boid loop
        for boid in boids:
            boid.draw(screen)
            boid.update(boids, a, c, s)

        # Update the screen
        pg.display.flip()
        clock.tick(60)
    pg.quit()

## Exercise 1

As a first exercise, we will run a simple 2D Boids simulator, based on the Reynolds' flocking rules we have seen during the lectures. Although this exercise is not strictly related to PSO, it provides a good source of inspiration (and intuition) on how PSO works. 

The simulator allows you to change various aspects of the simulation, specifically the total number of boids, the number of neighbors whose information is collected by each boid (to determine cohesion, alignment, and separation), and the relative weights of each of the 3 flocking rules (behavior coefficients). Spend some time with the simulator, and try different simulation configurations.

To help you figure out the behavior of the boids, you can find below the implementation of the `boid` class extracted from the source code of the simulator. In particular, check the `update` method.

- What is the effect of each behavior coefficient?
- Which combination of coefficients leads to the most ``natural'' flock behavior? 

In [ ]:
"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""
num_boids = 150  # advice: for graphical reasons avoid to use num_boids > 400
separation = 0.2  # default: 0.8
cohesion = 0.7  # default: 0.1
alignment = 2  # default: 1.6


# make sure you are calling the right version of python in the process below
os.popen(run(WIDTH, HEIGHT, num_boids, alignment, cohesion, separation)).read()

## Exercise 2

In this exercise we will perform a comparative analysis of the results of Genetic Algorithm (as seen in Lab 2), Evolution Strategies (as seen in Lab 3) and Particle Swarm Optimization. 
The script will perform a single run of GA, ES and PSO, on one of the benchmark functions we have seen in the previous labs. As usual, the algorithm parametrization is shown in the code and can be easily modified.

Questions:
-  What kind of behavior does PSO have on different benchmark functions (change the parameter `args["problem_class"]` to try at least a couple of functions), in comparison with the EAs? Does it show better or worse results? Does it converge faster or not?
- What happens if you run the script multiple times? Do the various algorithms (and especially PSO) show consistent behavior?
- Increase the problem dimensionality (`num_vars`, by default set to 2), e.g. to 10 or more. What do you observe in this case?
-  Change the population size (by changing`args["pop_size"]`, by default set to 50) and the number of generations (by changing `args["max_generations"]`, by default set to 100), such that their product is fixed (e.g. $50 \times 100$, $100 \times 100$, etc.). Try two or three different combinations and observe the behavior of the three different algorithms. What do you observe in this case? Is it better to have smaller/larger populations or a smaller/larger number of generations? Why?


In [ ]:
# -*- coding: utf-8 -*-

from inspyred import benchmarks

from utils.inspyred_utils import NumpyRandomWrapper
import utils.ga as ga
import utils.es as es
import utils.pso as pso
import matplotlib.pyplot as plt

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

num_vars = 100  # Number of dimensions of the search space

args = {}

# the problem class
args["problem_class"] = benchmarks.Rosenbrock

# other problems to try,
# see  https://pythonhosted.org/inspyred/reference.html#module-inspyred.benchmarks

# unimodal
# benchmarks.Sphere
# benchmarks.Rosenbrock

# multimodal
# benchmarks.Griewank
# benchmarks.Ackley
# benchmarks.Rastrigin
# benchmarks.Schwefel

# common parameters
args["max_generations"] = 100  # Number of generations
args["pop_size"] = 50  # population size

# parameters for the GA
args["gaussian_stdev"] = 0.5  # Standard deviation of the Gaussian mutations
args["mutation_rate"] = 0.5  # fraction of loci to perform mutation on
args["tournament_size"] = 2
args["num_elites"] = 1  # number of elite individuals to maintain in each gen

# parameters for the ES
args["num_offspring"] = 100  # lambda
args["sigma"] = 1.0  # default standard deviation
args["strategy_mode"] = es.INDIVIDUAL  # es.GLOBAL, es.INDIVIDUAL, None
args["mixing_number"] = 1  # rho

# parameters for the PSO
args["topology"] = pso.RING  # pso.RING, pso.STAR
args["neighborhood_size"] = 5  # used only for the ring topology
args["inertia"] = 0.5
args["cognitive_rate"] = 2.1
args["social_rate"] = 2.1

"""
-------------------------------------------------------------------------
"""

display = True  # Plot initial and final populations

rng = NumpyRandomWrapper(seed=41)

# Run GA
args["fig_title"] = "GA"
best_individual, best_fitness, final_pop = ga.run_ga(
    rng, num_vars=num_vars, display=display, use_log_scale=True, **args
)
print("Best GA fitness:", best_fitness)

# Run ES
args["fig_title"] = "ES"
best_individual, best_fitness, final_pop = es.run_es(
    rng, num_vars=num_vars, display=display, use_log_scale=True, **args
)
print("Best ES fitness:", best_fitness)

# Run PSO
args["fig_title"] = "PSO"
best_individual, best_fitness, final_pop = pso.run_pso(
    rng, num_vars=num_vars, display=display, use_log_scale=True, **args
)
print("Best PSO fitness:", best_fitness)

plt.ioff()
plt.show()

## Exercise 3 (Optional)

Now that you have some intuition on how an existing implementation of PSO works (as well as the underlying biological inspiration), you might find useful implementing a PSO algorithm on your own (in Python or any other language of your choice). As we have seen in the lecture, this can be coded in just a few lines. If you want, try to implement a **simple** version of PSO (with the parametrization found in the previous exercise), to minimize for instance the Sphere function. You could also try to include some constraints and a constraint handling technique.


## Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 

- When do you think it is useful to have a lower (higher) cognitive learning rate? What about the social learning rate?
- From a biological point of view, which neighborhood topology do you consider as the most plausible?
